## Lemma 2
Given two subsequences $s$ and $s'$ with identical aircraft in the same order, differing only in the takeoff times, if $t_x \le t'_x$ for all aircraft $x$ in $s$ and $s'$, the violation of an aircraft’s hard time window in $s$ will be no worse than its violation in $s'$, and hence if sequence $s$ is infeasible, so will be sequence $s'$.

In [6]:
from z3 import *
from math import pow

s = Solver()

δ = Function('δ', IntSort(), IntSort(), RealSort())
R = Function('R', IntSort(), RealSort())
B = Function('B', IntSort(), RealSort())

# --- aircraft etc as before ---
p1, a, p2, b, p3 = Ints('p1 a p2 b p3')
ac = [p1, a, p2, b, p3]

x, y = Ints('x y')
s.add(ForAll([x], R(x) >= 0))
s.add(ForAll([x], B(x) >= 0))
s.add(ForAll([x], R(x) >= B(x)))
s.add(ForAll([x, y], δ(x, y) >= 0))

# δ-identical a/b
for x_ in ac:
    s.add(δ(a, x_) == δ(b, x_))
    s.add(δ(x_, a) == δ(x_, b))



α = 1.0  # keep as float for pow; α >= 1 ensures convexity

# --- helpers: zmax, zmax_list, compute_T (unchanged) ---
def zmax(x, y): return If(x >= y, x, y)
def zmax_list(xs):
    m = xs[0]
    for x in xs[1:]:
        m = If(x >= m, x, m)
    return m

def compute_T(seq):
    T = {seq[0]: R(seq[0])}
    for i in range(1, len(seq)):
        preds = seq[:i]
        T[seq[i]] = zmax(R(seq[i]), zmax_list([T[x] + δ(x, seq[i]) for x in preds]))
    return T

S1, S2 = [p1, a, p2, b, p3], [p1, b, p2, a, p3]
T1, T2 = compute_T(S1), compute_T(S2)

# --- piecewise linear over-approx of d^α on [0, Dmax] ---
Dmax = 10.0  # choose a safe bound for lateness d = T[x]-B(x); increase if needed
# Breakpoints (denser -> tighter bound). Must cover [0, Dmax].
brks = [0.0, 1.0, 2.0, 4.0, 6.0, 10.0]

def add_power_overapprox(s, d, phi, brks, alpha):
    """
    Constrain phi >= d^alpha via chords on each [a,b] and ensure d lies in [min(brks), max(brks)].
    """
    a0, bN = brks[0], brks[-1]
    s.add(d >= RealVal(a0), d <= RealVal(bN))

    # For each interval, if d is in [a,b], enforce phi >= chord(a,b)(d)
    for a, b in zip(brks[:-1], brks[1:]):
        if b == a:
            continue
        m = (pow(b, alpha) - pow(a, alpha)) / (b - a)
        c = pow(a, alpha) - m * a
        s.add(Implies(And(d >= RealVal(a), d <= RealVal(b)),
                      phi >= RealVal(m) * d + RealVal(c)))

    # Optional: global safety using end tangents (keeps phi above curve near edges)
    # Left tangent at a0
    mL = alpha * pow(a0, alpha - 1) if a0 > 0 else 0.0
    cL = pow(a0, alpha) - mL * a0
    s.add(phi >= RealVal(mL) * d + RealVal(cL))
    # Right tangent at bN
    mR = alpha * pow(bN, alpha - 1)
    cR = pow(bN, alpha) - mR * bN
    s.add(phi >= RealVal(mR) * d + RealVal(cR))

# --- replace the nonlinear power with auxiliaries ---
# For each aircraft, D[x] = T[x] - B(x); Phi[x] ≈ (D[x])^α (over-approx)
D1, Phi1 = {}, {}
D2, Phi2 = {}, {}

for x_ in ac:
    D1[x_] = Real(f"D1_{x_}")
    Phi1[x_] = Real(f"Phi1_{x_}")
    s.add(D1[x_] == T1[x_] - B(x_))
    s.add(D1[x_] >= 0)  # usual lateness domain; drop if negatives allowed
    add_power_overapprox(s, D1[x_], Phi1[x_], brks, α)

    D2[x_] = Real(f"D2_{x_}")
    Phi2[x_] = Real(f"Phi2_{x_}")
    s.add(D2[x_] == T2[x_] - B(x_))
    s.add(D2[x_] >= 0)
    add_power_overapprox(s, D2[x_], Phi2[x_], brks, α)

# --- costs using linearized power ---
W1 = Real('W1')
s.add(W1 > 0)

def delay_cost_phi(Phi_dict):
    return Sum([W1 * Phi_dict[x_] for x_ in ac])

Dcost1 = delay_cost_phi(Phi1)
Dcost2 = delay_cost_phi(Phi2)

# your ordering premises etc.
s.add(R(a) <= R(b), B(a) <= B(b))

# Compare using over-approximated costs (sound for proving D1 > D2 is impossible)
s.add(Dcost1 > Dcost2)

# --- check ---
print(s.check())
if s.check() == sat:
    print("\nCounterexample found (with over-approx):\n")
    m = s.model()
    for d in m.decls():
        print(f"{d.name()} = {m[d]}")
elif s.check() == unsat:
    print("\nUNSAT — inequality cannot hold under the over-approx (sound proof on this domain)\n")
else:
    print("unknown")


sat

Counterexample found (with over-approx):

Phi1_p3 = 1/2
p1 = 3
Phi2_p3 = 0
Phi1_a = 1/8
Phi2_p1 = 0
Phi2_a = 1/2
Phi1_b = 1/8
Phi1_p1 = 0
Phi1_p2 = 1/8
p2 = 5
Phi2_b = 0
Phi2_p2 = 1/4
b = 4
W1 = 1/8
p3 = 6
a = 2
D2_p1 = 0
D2_p2 = 1/4
D1_a = 1/8
D2_p3 = 0
D2_b = 0
D2_a = 1/2
D1_b = 1/8
D1_p1 = 0
D1_p2 = 1/8
D1_p3 = 0
R = [else -> B(Var(0)) + R!46(Var(0))]
R!46 = [else ->
 If(Or(And(Not(Var(0) == 2),
           Not(Var(0) == 4),
           Not(Var(0) == 6),
           Not(Var(0) == 5)),
       And(Not(Var(0) == 2), Var(0) == 4),
       Var(0) == 2,
       And(Not(Var(0) == 2), Not(Var(0) == 4), Var(0) == 6),
       And(Not(Var(0) == 2),
           Not(Var(0) == 4),
           Not(Var(0) == 6),
           Var(0) == 5)),
    0,
    56)]
B = [else ->
 -1*R!46(Var(0)) +
 If(And(Not(Var(0) == 2), Not(Var(0) == 4), Var(0) == 6),
    38689,
    If(And(Not(Var(0) == 2), Var(0) == 4),
       1/4,
       If(Or(And(Not(Var(0) == 2),
                 Not(Var(0) == 4),
                 Not(Var(0